Graded Challenge 3<br>
Nama: Risqi Wahyu Permana<br>
Batch: HCK 6

## Library

In [1]:
import pandas as pd
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "hactiv8-risqi" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


##PROBLEM STATEMENT<br>
Lambatnya Peningkatan Penjualan dan pendapatan pada kuartal 1 hingga 3 pada tahun 2022.

## Penjabaran Masalah

### Problem 1
1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [2]:
df_complete = client.query('''
SELECT
  status,
  COUNT(status) as Jumlah, #Hitung jumlah
  EXTRACT(MONTH from created_at) as bulan, #Ambil Bulan
  EXTRACT(QUARTER from created_at) as quarter #Ambil Quarter
FROM
  `bigquery-public-data.thelook_ecommerce.order_items`
WHERE
  EXTRACT(YEAR FROM created_at) = 2022 AND #Filter pada tahun 2022
  EXTRACT(QUARTER from created_at) IN (1,2,3) AND #Filter pada kuartal 1, 2 dan 3
  status = 'Complete'
GROUP BY status, bulan, quarter
ORDER BY bulan
''').to_dataframe()

In [3]:
df_complete

,status,Jumlah,bulan,quarter
0,Complete,895,1,1
1,Complete,900,2,1
2,Complete,1160,3,1
3,Complete,1089,4,2
4,Complete,1209,5,2
5,Complete,1223,6,2
6,Complete,1320,7,3
7,Complete,1353,8,3
8,Complete,1466,9,3


#### Insight

Dapat dilihat jumlah transaksi setiap bulannya mengalami kenaikan yang konstan namun pertambahan dari bulan ke bulan hanya bertambah sedikit

### Problem 2
2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [4]:
df_total_penjualan = client.query('''
SELECT
  order_items.status,
  CAST(SUM(order_items.sale_price * orders.num_of_item) AS STRING FORMAT '$999,999') as #di cast ke usd
  total_penjualan,
  EXTRACT(MONTH from order_items.created_at) as Bulan,
  EXTRACT(QUARTER from order_items.created_at) as Quarter
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` as order_items
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.orders` as orders
on
  order_items.order_id = orders.order_id
WHERE
  EXTRACT(YEAR FROM order_items.created_at) = 2022 AND
  EXTRACT(QUARTER from order_items.created_at) IN (1,2,3) AND
  order_items.status = 'Complete'
GROUP BY status, Bulan, Quarter
ORDER BY Bulan
''').to_dataframe()

In [5]:
df_total_penjualan

,status,total_penjualan,Bulan,Quarter
0,Complete,"$86,474",1,1
1,Complete,"$96,763",2,1
2,Complete,"$139,392",3,1
3,Complete,"$126,231",4,2
4,Complete,"$138,402",5,2
5,Complete,"$142,820",6,2
6,Complete,"$147,791",7,3
7,Complete,"$151,784",8,3
8,Complete,"$177,691",9,3


#### Insight

Jika dilihat dari jumlah transaksi yang mengalami kenaikan tentu saja akan selaras dengan total penjualan yang akan juga naik dengan konstan setiap bulannya. Namun kembali lagi, walaupun mengalami kenaikan namun kenaikaannya sedikit setiap bulannya.

### Problem 3
3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [6]:
df_user = client.query('''
SELECT
  order_items.status,
  COUNT(distinct order_items.user_id) as jumlah_user,
  EXTRACT(MONTH from order_items.created_at) as bulan,
  EXTRACT(QUARTER from order_items.created_at) as quarter
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` as order_items
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.orders` as orders
on
  order_items.order_id = orders.order_id
WHERE
  EXTRACT(YEAR FROM order_items.created_at) = 2022 AND
  EXTRACT(QUARTER from order_items.created_at) IN (1,2,3)
  AND order_items.status = 'Complete'
GROUP BY status, bulan, quarter
ORDER BY bulan
''').to_dataframe()

In [7]:
df_user

,status,jumlah_user,bulan,quarter
0,Complete,645,1,1
1,Complete,645,2,1
2,Complete,787,3,1
3,Complete,764,4,2
4,Complete,843,5,2
5,Complete,850,6,2
6,Complete,925,7,3
7,Complete,966,8,3
8,Complete,1028,9,3


#### Insight

Berdasarkan data di atas dapat dilihat ternyata jumlah user juga meningkat setiap bulannya. Namun jumlah user yang bertambah setiap bulannya tidak banyak (bertahap - tahap).

### Problem 4
4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [8]:
df_kategori_terjual_terbanyak = client.query('''
SELECT
  tiap_cat.kategori,
  tiap_cat.jumlah_kategori,
  tiap_cat.bulan
FROM (
  SELECT #Sub query 1 untuk mendapatkan jumlah terjual dari masing kategori dan menampilkan nama kategorinya
      products.category as kategori,
      EXTRACT(MONTH FROM order_items.created_at) as bulan,
      COUNT(products.category) AS jumlah_kategori,
      EXTRACT(QUARTER FROM order_items.created_at) AS quarter
    FROM
      `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    JOIN
      `bigquery-public-data.thelook_ecommerce.products` AS products
    ON
      products.id = order_items.product_id
    JOIN
      `bigquery-public-data.thelook_ecommerce.orders` AS orders
    ON
      order_items.order_id = orders.order_id
    WHERE
      EXTRACT(YEAR FROM order_items.created_at) = 2022
      AND EXTRACT(QUARTER FROM order_items.created_at) IN (1, 2, 3)
      AND order_items.status = 'Complete'
    GROUP BY
      products.category, quarter, bulan
    ORDER BY bulan
)
AS tiap_cat
INNER JOIN (
  SELECT #Sub Query 2 untuk mendapatkan jumlah kategori terjual tebanyak setiap bulannya
    bulan,
    MAX(jumlah_kategori) AS max_jumlah_kategori
  FROM (
    SELECT #Subquery untuk mendapatkan jumlah terjual dari masing kategori tanpa menampilkan nama kategori
      EXTRACT(MONTH FROM order_items.created_at) as bulan,
      COUNT(products.category) AS jumlah_kategori,
      EXTRACT(QUARTER FROM order_items.created_at) AS quarter
    FROM
      `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    JOIN
      `bigquery-public-data.thelook_ecommerce.products` AS products
    ON
      products.id = order_items.product_id
    JOIN
      `bigquery-public-data.thelook_ecommerce.orders` AS orders
    ON
      order_items.order_id = orders.order_id #Untuk menampilkan kategori apa saja yang paling laku tiap bulannya dengan mencocokan antara sub query pertama dan sub query ke dua
    WHERE
      EXTRACT(YEAR FROM order_items.created_at) = 2022
      AND EXTRACT(QUARTER FROM order_items.created_at) IN (1, 2, 3)
      AND order_items.status = 'Complete'
    GROUP BY
      products.category, quarter, bulan
    order by quarter
  ) AS subquery
  GROUP BY
    bulan
)
AS max_pendapatan
ON
  tiap_cat.bulan = max_pendapatan.bulan AND
  tiap_cat.jumlah_kategori = max_pendapatan.max_jumlah_kategori
ORDER BY
  tiap_cat.bulan


''').to_dataframe()

In [9]:
df_kategori_terjual_terbanyak

,kategori,jumlah_kategori,bulan
0,Intimates,70,1
1,Tops & Tees,70,1
2,Intimates,72,2
3,Tops & Tees,93,3
4,Swim,81,4
5,Intimates,91,5
6,Sweaters,91,5
7,Intimates,94,6
8,Jeans,94,6
9,Tops & Tees,96,7


#### Insight

Dapat dilihat ada beberapa kategori produk yang memiliki jumlah kategori terjual yang sama. Namun yang tertinggi adalah `Sleep & Lounge` pada bulan ke 9.

### Problem 5
5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [10]:
df_pendapatan = client.query('''
SELECT
  tiap_cat.kategori,
  CAST(tiap_cat.total_keuntungan AS STRING FORMAT '$999,999') as pendapatan,
  tiap_cat.bulan,
  tiap_cat.quarter
FROM (
  SELECT #Sub query 1 untuk mendapatkan jumlah pendapatan dari masing kategori dan menampilkan nama kategorinya
      products.category as kategori,
      EXTRACT(MONTH FROM order_items.created_at) as bulan,
      SUM((order_items.sale_price  * orders.num_of_item)) AS total_keuntungan,
      EXTRACT(QUARTER FROM order_items.created_at) AS quarter
    FROM
      `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    JOIN
      `bigquery-public-data.thelook_ecommerce.products` AS products
    ON
      products.id = order_items.product_id
    JOIN
      `bigquery-public-data.thelook_ecommerce.orders` AS orders
    ON
      order_items.order_id = orders.order_id
    WHERE
      EXTRACT(YEAR FROM order_items.created_at) = 2022
      AND EXTRACT(QUARTER FROM order_items.created_at) IN (1, 2, 3)
      AND order_items.status = 'Complete'
    GROUP BY
      products.category, quarter, bulan
    ORDER BY bulan
)
AS tiap_cat
INNER JOIN (
  SELECT
    bulan,
    MAX(total_keuntungan) AS max_jumlah_keuntungan #Sub Query 2 untuk mendapatkan jumlah kategori dengan pendapatan terbanyak setiap bulannya
  FROM (
    SELECT
      EXTRACT(MONTH FROM order_items.created_at) as bulan,
      SUM((order_items.sale_price  * orders.num_of_item)) AS total_keuntungan, #Subquery untuk mendapatkan jumlah pendapatan dari masing kategori tanpa menampilkan nama kategori
      EXTRACT(QUARTER FROM order_items.created_at) AS quarter
    FROM
      `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
    JOIN
      `bigquery-public-data.thelook_ecommerce.products` AS products
    ON
      products.id = order_items.product_id
    JOIN
      `bigquery-public-data.thelook_ecommerce.orders` AS orders
    ON
      order_items.order_id = orders.order_id #Untuk menampilkan kategori apa saja yang paling untung tiap bulannya dengan mencocokan antara sub query pertama dan sub query ke dua
    WHERE
      EXTRACT(YEAR FROM order_items.created_at) = 2022
      AND EXTRACT(QUARTER FROM order_items.created_at) IN (1, 2, 3)
      AND order_items.status = 'Complete'
    GROUP BY
      products.category, quarter, bulan
    order by quarter
  ) AS subquery
  GROUP BY
    bulan
)
AS max_pendapatan
ON
  tiap_cat.bulan = max_pendapatan.bulan AND
  tiap_cat.total_keuntungan = max_pendapatan.max_jumlah_keuntungan
ORDER BY
  tiap_cat.bulan

''').to_dataframe()

In [11]:
df_pendapatan

,kategori,pendapatan,bulan,quarter
0,Jeans,"$8,869",1,1
1,Outerwear & Coats,"$15,221",2,1
2,Jeans,"$16,433",3,1
3,Outerwear & Coats,"$17,609",4,2
4,Outerwear & Coats,"$15,792",5,2
5,Jeans,"$19,095",6,2
6,Jeans,"$21,383",7,3
7,Outerwear & Coats,"$19,763",8,3
8,Outerwear & Coats,"$28,325",9,3


#### Insight

walaupun jumlah kategori dari `Sleep & Lounge` paling tinggi pada bulan 9. Namun pendapatan paling terbanyak berasal dari kategori `Outerwear & Coats` dengan jumlah pendapatan `$28,325`.

### Problem 6
6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan?

In [12]:
df_total_penjualan = client.query('''
SELECT
  order_items.status,
  SUM(order_items.sale_price * orders.num_of_item) as
  total_penjualan,
  EXTRACT(MONTH from order_items.created_at) as Bulan,
  EXTRACT(QUARTER from order_items.created_at) as Quarter
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` as order_items
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.orders` as orders
on
  order_items.order_id = orders.order_id
WHERE
  EXTRACT(YEAR FROM order_items.created_at) = 2022 AND
  EXTRACT(QUARTER from order_items.created_at) IN (1,2,3) AND
  order_items.status = 'Complete'
GROUP BY status, Bulan, Quarter
ORDER BY Bulan
''').to_dataframe()

In [13]:
# Buat dataframe df_cor_user, df_cor_transaksi, dan df_cor_jumlah
df_cor_user = df_user[['jumlah_user']]
df_cor_penjualan = df_total_penjualan[['total_penjualan']]
df_cor_transaksi = df_complete[['Jumlah']]

In [14]:
#Menggabungkan dataframe untuk dapat menggunakan method .corr() dari pandas
df_cor_user_transaksi = pd.concat([df_cor_user, df_cor_transaksi], axis=1)
df_cor_user_penjualan = pd.concat([df_cor_user, df_cor_penjualan], axis=1)

In [15]:
#Menghitung korelasi
print("Korelasi antara df_cor_user dan df_cor_transaksi:", df_cor_user_transaksi.corr())
print()
print("Korelasi antara df_cor_user dan df_cor_user_penjualan:", df_cor_user_penjualan.corr())

Korelasi antara df_cor_user dan df_cor_transaksi:              jumlah_user   Jumlah
jumlah_user      1.00000  0.99489
Jumlah           0.99489  1.00000

Korelasi antara df_cor_user dan df_cor_user_penjualan:                  jumlah_user  total_penjualan
jumlah_user         1.000000         0.959785
total_penjualan     0.959785         1.000000


#### Insight

Seharusnya korelasinya `sama saja` antara keduanya. Jika jumlah transaksi mengalami kenaikan maka total penjualan akan mengalami kenaikan pula. Hal tersebut dapat dilihat dari kedua uji korelasi di atas yang dimana masing - masing mendekati `1`.

## Poin Pertanyaan
1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

**Problem Statement** = Lambatnya Peningkatan Penjualan dan pendapatan pada kuartal 1 hingga 3 pada tahun 2022.

## Jawaban

1. Iya dapat terukur. Kita bisa melihat bahwa pertumbuhan jumlah user yang lambat mempengaruhi jumlah pendapatan dan juga jumlah produk yang laku.
2. Kurang masuk akal untuk menargetkan pendapatan sebesar `$250,000` pada kuartal 4 awal. Karena pada kuartal 3 akhir pendapatan yang didapatkan sebesar `$177,691`. Kemungkinan target tersebut baru dapat tercapai pada pertengahan atau akhir dari kuartal 4. Jika memaksa untuk mencapai target tersebut mungkin harus melakukan perubahan dalam pemasaran atau promosi produk.
3. Menurut saya perusahan layak untuk mendapatkan investor baru. Hal ini dikarenakan jumlah user yang terus bertambah dengan konstan selaras dengan jumlah transaksi dan total penjualan, walaupun bertambah dengan lambat. Dengan mendapatkan investor baru, perusahaan dapat mengembangkan modal tambahan untuk meningkatkan upaya pemasaran dan kualitas layanan untuk mendapatkan pelanggan baru.